In [1]:
!pip install pyspark==3.5.0

In [2]:
!pip install findspark

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import findspark
findspark.init()

In [5]:
# Conectando ao google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reducao de dimensionalidade usando colunas artificiais

In [6]:
spark = SparkSession.builder.appName("PCA").getOrCreate()

In [7]:
from pyspark.ml.feature import PCA

In [8]:
carros = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Spark/datas/Carros.csv", header=True, inferSchema=True, sep=";")

In [9]:
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [10]:
from pyspark.ml.feature import VectorAssembler
vectas = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas", "RelEixoTraseiro", "Peso",
                                    "Tempo", "TipoMotor", "Transmissao", "Marchas", "Carburadors"], outputCol="caracteristicas")
carros = vectas.transform(carros)
carros.select("caracteristicas").show(truncate=False)

+-----------------------------------------------------+
|caracteristicas                                      |
+-----------------------------------------------------+
|[21.0,6.0,160.0,39.0,262.0,1646.0,0.0,1.0,4.0,4.0]   |
|[21.0,6.0,160.0,39.0,2875.0,1702.0,0.0,1.0,4.0,4.0]  |
|[228.0,4.0,108.0,385.0,232.0,1861.0,1.0,1.0,4.0,1.0] |
|[214.0,6.0,258.0,308.0,3215.0,1944.0,1.0,0.0,3.0,1.0]|
|[187.0,8.0,360.0,315.0,344.0,1702.0,0.0,0.0,3.0,2.0] |
|[181.0,6.0,225.0,276.0,346.0,2022.0,1.0,0.0,3.0,1.0] |
|[143.0,8.0,360.0,321.0,357.0,1584.0,0.0,0.0,3.0,4.0] |
|[244.0,4.0,1467.0,369.0,319.0,20.0,1.0,0.0,4.0,2.0]  |
|[228.0,4.0,1408.0,392.0,315.0,229.0,1.0,0.0,4.0,2.0] |
|[192.0,6.0,1676.0,392.0,344.0,183.0,1.0,0.0,4.0,4.0] |
|[178.0,6.0,1676.0,392.0,344.0,189.0,1.0,0.0,4.0,4.0] |
|[164.0,8.0,2758.0,307.0,407.0,174.0,0.0,0.0,3.0,3.0] |
|[173.0,8.0,2758.0,307.0,373.0,176.0,0.0,0.0,3.0,3.0] |
|[152.0,8.0,2758.0,307.0,378.0,18.0,0.0,0.0,3.0,3.0]  |
|[104.0,8.0,472.0,293.0,525.0,1798.0,0.0,0.0,3.0

In [11]:
pca = PCA(k=3, inputCol="caracteristicas", outputCol="caracteristicasPCA")
modelo = pca.fit(carros)

In [13]:
# transformacao
resultado = modelo.transform(carros)
resultado.select("caracteristicasPCA").show(truncate=False)

+-----------------------------------------------------------+
|caracteristicasPCA                                         |
+-----------------------------------------------------------+
|[618.7707206779619,-937.7123949973537,1231.9633529945518]  |
|[3112.9887675342234,-161.05746385491454,1191.8619913054386]|
|[640.49590077107,-1120.7188865110413,1320.0756315189058]   |
|[3466.09568775567,-149.6942141829825,1401.2041780368531]   |
|[661.4577445758738,-812.4592128844109,1395.294932831636]   |
|[769.2343671787745,-1120.4160559477314,1518.7436632279532] |
|[644.8369503533219,-727.9539376169612,1313.6815210979355]  |
|[9.101880661709954,1061.2954036677907,1045.1710500215688]  |
|[67.1336096650841,878.4793682045025,1143.9379120496162]    |
|[31.390504477140738,1095.3694498285759,1306.0124861906324] |
|[32.891659222089714,1091.1521230845244,1310.0881577350904] |
|[-118.37273751675384,1832.7719274058172,2088.6955393326043]|
|[-150.18148405358014,1820.8730926512799,2091.103355076612] |
|[-184.0